In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install the 'timm' library for working with pre-trained models and 'faiss-gpu' for GPU-accelerated similarity search.

!pip install timm faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00


In [3]:
# Standard Libraries
import time
import os
import copy

# NumPy and Pandas
import numpy as np
import pandas as pd

# PyTorch and related modules
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

# torchvision for image-related operations
from torchvision import transforms, datasets
import torchvision

import matplotlib.pyplot as plt

# timm for working with pre-trained models
import timm

In [4]:
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("timm version:", timm.__version__)
print("PyTorch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

NumPy version: 1.23.5
Pandas version: 1.5.3
timm version: 0.9.7
PyTorch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118


In [5]:
!unzip '/content/drive/MyDrive/DL Labs/dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_099.jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_100.jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_101 (2).jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_101.jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_102 (2).jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_102.jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_103 (2).jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_103.jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_104 (2).jpg  
  inflating: emirhan_human_dataset/datasets/human_data/train_data/running/images_104 (3).jpg  
  inflating: emirha

In [6]:
transformations = {
    'train': transforms.Compose([
        # Randomly resize and crop the image to 224x224 pixels
        transforms.RandomResizedCrop(224),
        # Randomly flip the image horizontally for data augmentation
        transforms.RandomHorizontalFlip(),
        # Convert the image to a PyTorch tensor
        transforms.ToTensor(),
        # Normalize the image with mean and standard deviation values
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_directory = '../content/emirhan_human_dataset/datasets/human_data/'

datasets = {
    'train': datasets.ImageFolder(os.path.join(data_directory, 'train_data'), transformations['train']),
    'test': datasets.ImageFolder(os.path.join(data_directory, 'test_data'), transformations['test'])
}


In [7]:
dataloaders = {
    'train': torch.utils.data.DataLoader(datasets['train'], batch_size=4, shuffle=True, num_workers=4),
    'test': torch.utils.data.DataLoader(datasets['test'], batch_size=4, shuffle=True, num_workers=4)
}

dataset_sizes = {
    'train': len(datasets['train']),
    'test': len(datasets['test'])
}

class_names = datasets['train'].classes


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [9]:

def train_model(model, loss_fn, optimizer, scheduler, num_epochs=25):
    start_time = time.time()
    best_model_weights = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch_inputs, batch_labels in dataloaders[phase]:
                batch_inputs = batch_inputs.to(device)
                batch_labels = batch_labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                # Track history if only in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    batch_outputs = model(batch_inputs)
                    _, batch_preds = torch.max(batch_outputs, 1)
                    batch_loss = loss_fn(batch_outputs, batch_labels)

                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        batch_loss.backward()
                        optimizer.step()

                running_loss += batch_loss.item() * batch_inputs.size(0)
                running_corrects += torch.sum(batch_preds == batch_labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_accuracy = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_accuracy:.4f}')

            # Deep copy the model
            if phase == 'test' and epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                best_model_weights = copy.deepcopy(model.state_dict())

        print()

    elapsed_time = time.time() - start_time
    print(f'Training complete in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s')
    print(f'Best Validation Acc: {best_accuracy:.4f}')
    # Load best model weights
    model.load_state_dict(best_model_weights)
    return model


In [10]:
#create an EfficientNetV2 model and move it to the specified device:
efficientnet_model = timm.create_model("tf_efficientnetv2_s_in21ft1k", pretrained=True, drop_rate=0.3, num_classes=len(class_names))
efficientnet_model = efficientnet_model.to(device)


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21ft1k to current tf_efficientnetv2_s.in21k_ft_in1k.
  model = create_fn(


In [11]:
loss_criterion = nn.CrossEntropyLoss() # Define the loss function (criterion)
optimizer_efficientnet = optim.SGD(efficientnet_model.parameters(), lr=0.001, momentum=0.9) # Specify the optimizer for training (Stochastic Gradient Descent - SGD)



In [12]:
learning_rate_scheduler = lr_scheduler.StepLR(optimizer_efficientnet, step_size=7, gamma=0.1) ## Decay the learning rate by a factor of 0.1 every 7 epochs



In [13]:
trained_efficientnet_model = train_model(efficientnet_model, loss_criterion, optimizer_efficientnet, learning_rate_scheduler, num_epochs=10)


Epoch 0/9
----------
Train Loss: 1.8786 Acc: 0.4280
Test Loss: 0.8653 Acc: 0.7350

Epoch 1/9
----------
Train Loss: 1.3165 Acc: 0.5909
Test Loss: 0.7846 Acc: 0.7613

Epoch 2/9
----------
Train Loss: 1.1685 Acc: 0.6375
Test Loss: 0.7604 Acc: 0.7760

Epoch 3/9
----------
Train Loss: 1.0733 Acc: 0.6701
Test Loss: 0.6921 Acc: 0.7970

Epoch 4/9
----------
Train Loss: 0.9940 Acc: 0.6916
Test Loss: 0.6521 Acc: 0.8090

Epoch 5/9
----------
Train Loss: 0.9561 Acc: 0.6999
Test Loss: 0.6567 Acc: 0.8003

Epoch 6/9
----------
Train Loss: 0.9106 Acc: 0.7177
Test Loss: 0.6289 Acc: 0.8183

Epoch 7/9
----------
Train Loss: 0.7919 Acc: 0.7452
Test Loss: 0.6041 Acc: 0.8233

Epoch 8/9
----------
Train Loss: 0.7518 Acc: 0.7645
Test Loss: 0.5955 Acc: 0.8250

Epoch 9/9
----------
Train Loss: 0.7501 Acc: 0.7611
Test Loss: 0.5868 Acc: 0.8253

Training complete in 57m 17s
Best Validation Acc: 0.8253


In [23]:
correct = 0
total = 0

# Disable gradient computation for this section as we're just testing the model
with torch.no_grad():
    # Loop through the test dataset
    for inputs, labels in dataloaders['test']:
        # Move inputs and labels to the device (CPU or GPU)
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = trained_efficientnet_model(inputs) # Forward pass to get predictions from the model

        _, predicted = torch.max(outputs.data, 1) # Get the index of the predicted class (class with the highest probability)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # Count the number of correct predictions by comparing with the true labels


accuracy = 100 * correct / total

print(f"Accuracy on test data: {accuracy:.2f}%")


Accuracy on test data: 82.53%
